<a href="https://colab.research.google.com/github/loujjj/Machine-Learning/blob/main/Example_Iris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Classification of Iris Species**

####**Importing the Necessary Libraries**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential

from sklearn import datasets
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

####**The three iris flower species in this example.**

#####**Accessing Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
ls

In [ ]:
cd gdrive/MyDrive/'Colab Notebooks'/Book

/content/gdrive/MyDrive/Colab Notebooks/Book


In [ ]:
ls

#####*Iris Setosa*

In [ ]:
im = plt.imread('Figures/iris-setosa.jpeg')
plt.imshow(im)

#####*Iris Versicolor*

In [ ]:
im = plt.imread('Figures/iris-versicolor.jpeg')
plt.imshow(im)

#####*Iris Virginica*

In [ ]:
im = plt.imread('Figures/iris-virginica.jpeg')
plt.imshow(im)

####**Importing and Preparing the Data (Option 1)**

#####**Importing the Data**

The Iris data set contains six columns.

In [ ]:
iris_dataset = pd.read_csv('Iris.csv', index_col=None)
print(iris_dataset.info())

We drop the first column, since it is just a row number.

In [ ]:
iris_dataset = iris_dataset.drop(['Id'],axis=1)
print(iris_dataset.info())

The first column correspond to the information about the sepal length (in cm), the second column about the sepal width (in cm), the third about the petal length (in cm), and the fourth the petal width (in cm). <br>
The fifth column represent the names of each of the flower species, the *Iris-setosa*, the *Iris-versicolor*, and the *Iris-virginica*.

In [ ]:
print(iris_dataset.head(10))

#####**Preparing the Data**

The first four columns will become our input, which we will assign to the variable $X$.

In [ ]:
X = iris_dataset.loc[:,iris_dataset.columns != 'Species']
print(X)

The last column will become our label, which we will assign to the variable $y$.

In [ ]:
y_dataset = iris_dataset.loc[:,iris_dataset.columns == 'Species']
print(y_dataset)

We use the [LabelEncoder()](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html) function to encode target labels with values between 0 and $n-1$, where $n$ is the number of classes. The *fit_transform()* method returns encoded labels.

In [ ]:
y_label = LabelEncoder().fit_transform(y_dataset) 
print(y_label)

Then, we use the [tf.keras.utils.to_categorical()](https://www.tensorflow.org/api_docs/python/tf/keras/utils/to_categorical) function to convert the class vector, $y\_label$, to binary class matrix.

In [ ]:
y = tf.keras.utils.to_categorical(y_label)
print(y)

####**Importing and Preparing the Data (Option 2)**

#####**Importing the Data**

In [ ]:
iris_dataset = datasets.load_iris()
X = iris_dataset.data
y_label = iris_dataset.target

In [ ]:
print(X)

In [ ]:
y = tf.keras.utils.to_categorical(y_label)
print(y)

####**Defining the model**

We split the training data, validation data, and test data into 70%, 15%, and 15%, respectively.

In [ ]:
trainRatio = 70/100
valRatio = 15/100
testRatio = 15/100

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 1 - trainRatio)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size = testRatio/(testRatio + valRatio))

In [ ]:
print(X_train.shape)

In [ ]:
print(y_train.shape)

In [ ]:
print(y_test.shape)

(23, 3)


In [ ]:
model = Sequential([
    keras.layers.Input(shape = X_train.shape[1:]),
    keras.layers.Dense(4, activation='sigmoid'),
    keras.layers.Dense(3, activation='softmax')
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 4)                 20        
                                                                 
 dense_1 (Dense)             (None, 3)                 15        
                                                                 
Total params: 35
Trainable params: 35
Non-trainable params: 0
_________________________________________________________________


####**Compiling the model**

In [ ]:
model.compile(optimizer = 'adam',
              loss = keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

####**Training the model**

In [ ]:
history = model.fit(X_train, y_train, epochs=1000, validation_data=(X_val, y_val), verbose=1)

####**Displaying the weights and biases**

In [ ]:
for layer in model.layers:
  print(layer.name, layer)

In [ ]:
# the input-to-hidden layer weights
print(model.layers[0].weights)

In [ ]:
# the hidden-to-output layer weights
print(model.layers[1].weights)

In [ ]:
# the input-to-hidden layers bias
print(model.layers[0].bias.numpy())

In [ ]:
# the hidden-to-output layer bias
print(model.layers[1].bias.numpy())

####**Evaluating the performance**

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
pd.DataFrame(history.history).plot(figsize=(10,6))
plt.grid(True)
plt.gca().set_ylim(0,1)
plt.show()

In [ ]:
prediction = model.predict(X_val)
print(prediction)

In [ ]:
print(y_val)

In [ ]:
cm = confusion_matrix(y_test.argmax(axis=1), prediction.argmax(axis=1))

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()

In [ ]:
accuracy = 100*np.trace(cm)/np.sum(cm)
print(accuracy)